<h1 style='text-align:center'>Diabetes Dataset</h1>

Two datasets are used in this section: load_diabetes from sklearn toy dataset and the other one is prima indiands diabetes.
Pima Indiands: http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data

## 1. Toy dataset sklearn

In [1]:
import tensorflow as tf
import numpy as np

#### How does the data look?

In [2]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
X[:5], y[:5]

(array([[ 0.03807591,  0.05068012,  0.06169621,  0.02187235, -0.0442235 ,
         -0.03482076, -0.04340085, -0.00259226,  0.01990842, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, -0.02632783, -0.00844872,
         -0.01916334,  0.07441156, -0.03949338, -0.06832974, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, -0.00567061, -0.04559945,
         -0.03419447, -0.03235593, -0.00259226,  0.00286377, -0.02593034],
        [-0.08906294, -0.04464164, -0.01159501, -0.03665645,  0.01219057,
          0.02499059, -0.03603757,  0.03430886,  0.02269202, -0.00936191],
        [ 0.00538306, -0.04464164, -0.03638469,  0.02187235,  0.00393485,
          0.01559614,  0.00814208, -0.00259226, -0.03199144, -0.04664087]]),
 array([ 151.,   75.,  141.,  206.,  135.]))

In [3]:
import pandas as pd
data = pd.DataFrame(data=X, columns='age sex bmi map tc ldl hdl tch ltg glu'.split(' '))
data.head()

#Data is already scaled
#he dataset in scikit-learn is standarized (zero mean and unit L2 norm).
#y is quantitative measure of disease progression one year after baseline

,age,sex,bmi,map,tc,ldl,hdl,tch,ltg,glu
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [4]:
#Get min/max for each column
min_frame = data.min().to_frame()
max_frame = data.max().to_frame()

#rename columns
max_frame.columns = ['Max']
min_frame.columns = ['Min']

#merge columns
pd.concat([max_frame, min_frame], axis=1)

,Max,Min
age,0.110727,-0.107226
sex,0.050680,-0.044642
bmi,0.170555,-0.090275
map,0.132044,-0.112400
tc,0.153914,-0.126781
ldl,0.198788,-0.115613
hdl,0.181179,-0.102307
tch,0.185234,-0.076395
ltg,0.133599,-0.126097
glu,0.135612,-0.137767


#### Split training and test data

In [5]:
#shuffle and split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    diabetes.data, diabetes.target, test_size=0.25, random_state=42)

## Neural Network

In [6]:
#Graph

# NN variables
num_features = X_train.shape[1]
train_size = X_train.shape[0]
hidden_nodes = 5
y = tf.placeholder("float") #single value

# input layer
X = tf.placeholder(tf.float32, [None, num_features])

# hidden layer 1
w_1 = tf.Variable(tf.zeros([num_features, 12]))
b_1 = tf.Variable(tf.zeros([12]))
h_1 = tf.nn.sigmoid(tf.matmul(X, w_1) + b_1)

# hidden layer 2
w_2 = tf.Variable(tf.zeros([12, 8]))
b_2 = tf.Variable(tf.zeros([8]))
h_2 = tf.nn.sigmoid(tf.matmul(h_1, w_2) + b_2)

# output layer
w_3 = tf.Variable(tf.zeros([8, 1])) #one output
y_pred = tf.matmul(h_2, w_3)

# # Cost: Mean squared error
cost = tf.reduce_sum(tf.pow(y-y_pred, 2))/(train_size)
# cost = tf.reduce_sum(tf.pow(y_pred-y, 2))/(2*train_size)
# cost1 = tf.reduce_sum(tf.pow(y_pred-y, 2))/(2*xxx_size)

# # Gradient descent
updates = tf.train.GradientDescentOptimizer(0.003).minimize(cost)

In [8]:
#Run

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(300):
    # Train with each example
    for i in range(len(X_train)):
        sess.run(updates, feed_dict={X: X_train[i: i + 1], y: y_train[i: i + 1]})

    # Display logs per epoch step
    if (epoch + 1) % 50 == 0:
        c = sess.run(cost, feed_dict={X: X_test, y: y_test})
#         c1 = sess.run(cost, feed_dict={X: X_test[:2], y: y_test[:2]})
#         c = sess.run(cost, feed_dict={X: X_train, y: y_train})
#         print("Epoch:", '%04d' % (epoch + 1), "cost(tr)=", "{:.9f}".format(c))
            
        print("Epoch:", '%04d' % (epoch + 1), "cost(test)=", "{:.9f}".format(c))
#         print("COST X_test 0:", " cost(test)=", "{:.9f}".format(c1))
#     break

sess.close()

Epoch: 0050 cost(test)= 207279.859375000
Epoch: 0100 cost(test)= 207832.625000000
Epoch: 0150 cost(test)= 208559.234375000
Epoch: 0200 cost(test)= 208629.953125000
Epoch: 0250 cost(test)= 208637.031250000
Epoch: 0300 cost(test)= 208638.093750000


## Load Model, predict 1 element

#### Lets check what the difference is from a prediction and a true value for X_test[0]:

In [82]:
X_test[0], y_test[0]

(array([ 0.04534098, -0.04464164, -0.00620595, -0.01599922,  0.1250187 ,
         0.1251981 ,  0.019187  ,  0.03430886,  0.03243323, -0.0052198 ]),
 219.0)

## 2. Pima Indians

1. Number of times pregnant 
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test 
3. Diastolic blood pressure (mm Hg) 
4. Triceps skin fold thickness (mm) 
5. 2-Hour serum insulin (mu U/ml) 
6. Body mass index (weight in kg/(height in m)^2) 
7. Diabetes pedigree function 
8. Age (years) 
9. Class variable (0 or 1) 

In [1]:
import pandas as pd
import numpy as np

#### Load Data

In [2]:
names = ['times_pregnant', 'glucose', 'diastolic', 'tricep', 'serum_insulin', 'body_mass', 'pedigreeF', 'age', 'class']
data = pd.read_csv('data/pima-indians-diabetes.csv', names=names)

In [3]:
data[:5]

,times_pregnant,glucose,diastolic,tricep,serum_insulin,body_mass,pedigreeF,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### Split train test

In [6]:
#Split X,y
X = data.iloc[:, :8]
y = data['class']
# X = scalar.fit_transform(X)

In [7]:
#shuffle and split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [11]:
y_train[:5]

357    1
73     0
352    0
497    0
145    0
Name: class, dtype: int64

In [12]:
X_train[:5]

,times_pregnant,glucose,diastolic,tricep,serum_insulin,body_mass,pedigreeF,age
357,13,129,0,30,0,39.9,0.569,44
73,4,129,86,20,270,35.1,0.231,23
352,3,61,82,28,0,34.4,0.243,46
497,2,81,72,15,76,30.1,0.547,25
145,0,102,75,23,0,0.0,0.572,21


#### Prepare data for NN

In [10]:
#Prepare data for NN
X_train, y_train = X_train.as_matrix(), y_train.as_matrix()
X_test, y_test = X_test.as_matrix(), y_test.as_matrix()

In [14]:
# Scale train and test individually
# It's not allowed to scale X and then split.
# First split then scale, so the train and test are disjoint scalings
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.fit_transform(X_test)


In [17]:
X_test[:5]

array([[ 0.52134871, -0.74973601, -0.45176852,  0.70763362,  0.99686588,
         0.22908898, -0.16883507,  0.70770842],
       [-0.6097129 , -0.30495773,  0.32350617,  0.65028214, -0.69589897,
         0.4262795 , -1.09062651, -1.06114387],
       [-0.6097129 , -0.43203724, -0.17814216, -1.18496521, -0.69589897,
        -0.14209317, -1.05793887, -1.06114387],
       [ 1.08687952, -0.46380712,  0.55152814, -1.18496521, -0.69589897,
        -0.86125859,  1.22365839, -0.01591297],
       [ 0.80411412,  0.45751933,  1.00757207, -1.18496521, -0.69589897,
        -0.24648815, -0.88796314,  1.27052506]])

In [18]:
#one hot encode the y variable for NN
#each class needs one column
def one_encoded(X):
    res = []
    for i in X:
        res.append([i, -(i-1)])
    return res
        
y_train = np.array(one_encoded(y_train))
y_test = np.array(one_encoded(y_test))

In [19]:
#class 1 is [1, 0]
#class 0 is [0, 1]
y_train[:5]

array([[1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]])

#### Neural Network

In [20]:
import tensorflow as tf

In [21]:
# Helper functions
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape=shape, stddev=0.05))
def new_biases(length):
    return tf.constant(0.05, shape=[length])
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 activation):    # Type of activation

    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases

    if activation == 'Relu':
        layer = tf.nn.relu(layer)
    if activation == 'Sigmoid':
        layer = tf.nn.sigmoid(layer)

    return layer

In [22]:
# placeholders and variables
x = tf.placeholder(tf.float32, shape=[None, 8], name = 'input_x')
y_true = tf.placeholder(tf.float32, shape=[None, 2], name='y')
# y_true = tf.placeholder(tf.float32)

In [23]:
# Hidden Layers
layer_fc1 = new_fc_layer(x, 8, 12, activation='Relu')
layer_fc2 = new_fc_layer(layer_fc1, 12, 8, activation='Relu')

In [24]:
# Prediction
y_pred = new_fc_layer(layer_fc2, 8, 2, activation='None')

In [25]:
# Cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_pred, y_true))

In [26]:
# Optimize
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [27]:
# Performance Metric
# correct_prediction = tf.equal(tf.argmin(y_pred, 1), tf.argmin(y_true, 1))   
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [28]:
# y_test[:5]
X_train.shape

(576, 8)

In [29]:
#Run
batch_size = 10
training_epochs = 1000
display_step = 20

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X_train)/batch_size)
        X_batches = np.array_split(X_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # batch_y.shape = (batch_y.shape[0], 1)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y_true: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=","{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmin(y_pred, 1), tf.argmin(y_true, 1))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y_true: y_test}))
#     global result 
#     result = tf.argmax(y_pred, 1).eval({X: X_test, y: y_test})1
    result = tf.argmin(y_pred, 1) 
    print(result.eval(feed_dict={x: X_test[:20]}))

Epoch: 0001 cost= 0.694049877
Epoch: 0021 cost= 0.572389882
Epoch: 0041 cost= 0.491518956
Epoch: 0061 cost= 0.467381400
Epoch: 0081 cost= 0.451004713
Epoch: 0101 cost= 0.441876919
Epoch: 0121 cost= 0.436951418
Epoch: 0141 cost= 0.433364190
Epoch: 0161 cost= 0.430502170
Epoch: 0181 cost= 0.428433959
Epoch: 0201 cost= 0.426903761
Epoch: 0221 cost= 0.425484567
Epoch: 0241 cost= 0.423935158
Epoch: 0261 cost= 0.422420031
Epoch: 0281 cost= 0.421115068
Epoch: 0301 cost= 0.419939759
Epoch: 0321 cost= 0.418755901
Epoch: 0341 cost= 0.417413632
Epoch: 0361 cost= 0.416070496
Epoch: 0381 cost= 0.414602478
Epoch: 0401 cost= 0.413125103
Epoch: 0421 cost= 0.411731318
Epoch: 0441 cost= 0.410296634
Epoch: 0461 cost= 0.408693609
Epoch: 0481 cost= 0.407018275
Epoch: 0501 cost= 0.405441168
Epoch: 0521 cost= 0.403984454
Epoch: 0541 cost= 0.402568757
Epoch: 0561 cost= 0.401089223
Epoch: 0581 cost= 0.399694845
Epoch: 0601 cost= 0.398267872
Epoch: 0621 cost= 0.396905833
Epoch: 0641 cost= 0.395773200
Epoch: 066

In [30]:
y_test[:20]

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0]])